# 6. Conversion from cube map to sphere map
This is the final part of the workshop.  
Here we convert the cube map generated in the previous section to a sphere map which can be used in Mitaka.

First, load the cube map images and display them for check. Enter the following codes and run it.
```
import math
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

mpl.rcParams['figure.autolayout'] = 'false'

src_dir = './cube_map'
dest_dir = './sphere_map'

plane_name = ('posx', 'negx', 'posy', 'negy', 'posz', 'negz')

# Transform matrix (Galactic coordinates -> plane coordinates)
RT = np.empty((6,3,3))  # eR, eU, eF (ex, ey, -ez)
RT[0] = np.array([[-1, 0, 0], [ 0, 0, 1], [ 0,-1, 0]]).T # posx
RT[1] = np.array([[ 1, 0, 0], [ 0, 0, 1], [ 0, 1, 0]]).T # negx
RT[2] = np.array([[ 0,-1, 0], [-1, 0, 0], [ 0, 0, 1]]).T # posy
RT[3] = np.array([[ 0,-1, 0], [ 1, 0, 0], [ 0, 0,-1]]).T # negy
RT[4] = np.array([[ 0, 1, 0], [ 0, 0, 1], [-1, 0, 0]]).T # posz
RT[5] = np.array([[ 0,-1, 0], [ 0, 0, 1], [ 1, 0, 0]]).T # negz

N = 512

# make sure the directory 'dest_dir' is avairable
if not os.path.isdir(dest_dir):
    os.mkdir(dest_dir)
    
# Load cube map images
img = [None]*6
for ip in range(6):
    fn = '{0}/milkyway_map_{1}_{2}.png'.format(src_dir, N, plane_name[ip]) # Generate output filename
    img[ip] = Image.open(fn)

    
def preview_cube_map():
    plt.clf()
    plt.figure(figsize=(8,6))
    xt = yt = N/2

    fig_pos = (7, 5, 2, 10, 8, 6) # Position of subplot
    for ip in range(6):
        plt.subplot(3,4,fig_pos[ip])
        plt.axis('off') # removing axes
        plt.imshow(np.array(img[ip]))
        plt.text(xt,yt, plane_name[ip], ha='center', va='center', size=15, color='white')

    plt.subplots_adjust(0,0,1,1,0,0)
    plt.show()

    
preview_cube_map()
```

Then, let's generate a sphere map from the cube map.
In the sphere map, the $x$-axis is aligned to the Galactic plane and the Galactic center $l=0$ is located at the center of the map. The $y$-axis is related with the Galactic latitude $b$. If the $x$- and $y$- axes are normalized so that $0\le x<1$ and $0 \le y < 1$, the Galactic coordinates $(l, b)$ are given by the following relation:
$$
l = -2 \pi x + \pi,
\qquad
b = (2y - 1) \pi / 2.
$$
As already mentioned, with the Galactic coordinates $(l, b)$, their Cartesian coordinates are given as follows
$$
\begin{array}{l}
x_g = \cos(b) \cos(l)\\
y_g = \cos(b) \sin(l)\\
z_g = \sin(b).
\end{array}
$$
With this and the transformation matrices from the Galactic coordinates to the plane coordinates, we can obtain the pixel on the cube map for the coordinates of the sphere map $x$ and $y$ in a similar way that we generate the intensity maps.
Enter the following codes and run it to generate a sphere map.
```
# Generate sphere map
Nx = N*4
Ny = N*2
imgs = Image.new('RGB', (Nx,Ny), (0,0,0))

def sampling_cube_map(x,y):
    l = -x * 2 * math.pi + math.pi
    b = (2*y - 1) * 0.5 * math.pi
    cosb = math.cos(b)
    xg = cosb * math.cos(l)
    yg = cosb * math.sin(l)
    zg = math.sin(b)
    rg = np.array([xg,yg,zg])
    for ip in range(6):
        r = rg @ RT[ip]
        if r[2]<=0:
            continue
        X = r[0]/r[2]
        Y = r[1]/r[2]
        ix = math.floor(0.5*(X+1)*N)
        iy = math.floor(0.5*(Y+1)*N)
        if ix<0 or ix>= N or iy<0 or iy>= N:
            continue
        iyr = N - iy - 1
        (r,g,b) = img[ip].getpixel((ix,iyr))
        return r
    return 0


for iy in range(Ny):
    y = iy/Ny
    iyr = Ny - iy - 1
    if iy % 100 == 0:
        print('Processing line {0}...'.format(iy))
    for ix in range(Nx):
        x = ix/Nx
        val = sampling_cube_map(x,y)
        imgs.putpixel((ix,iyr), (val,val,val))
print('Completed')

plt.figure(figsize=(12,6))
plt.axis('off')
plt.imshow(imgs)
```

Finally, let's save the generated sphere map.
Enter the following code and run it to save it. Here, we save it in both PNG and JPEG format. The latter (`milkyway.jpg`) can be used in Mitaka. Copy it into the `textures` sub folder in the Mitaka folder and see the Milky Way generated by yourself.
```
# Save sphere map
fno = '{0}/milkyway_map_{1}.png'.format(dest_dir, N) # Generate output filename
imgs.save(fno)
# Save sphere map for Mitaka
fno = '{0}/milkyway.jpg'.format(dest_dir) # Generate output filename
imgs.save(fno)
```

Congratulations!  
You have finished the basic part of the workshop/practicum.

If you have any time left at this time, try the following extended practicum.

1. **Generate a shphere map for another index of i1.**First, copy the extracted data for the new index i1 (256 files except for i1=20) from the USB memory into your `extract_data` folder and then repeat the precedures in the sections 4, 5, and 6 for this i1. (The areas covered by observations for respective index i1 were visualized as images in the `area` folder. See these images for selecting the index i1.)

2. **Merge intensity maps for different index of i1.** First, write Python code for merging two set of the intensity maps and using it merge the intensity maps generated for i1=0 to those for the other i1. Then, repeat the sections 5 and 6 to make texture maps with wider observing area.

3. **Merge the intensity maps generated by all members of your group for different index of i1 and make texture maps from them.**

In the USB memory, there are the intensity maps generated using all the data of Gaia DR1 in the `intensity_maps_all` folder. You can make an entire Milky Way map with them by repeating the sections 5 and 6.